In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv('./kaggle/input/spaceship-titanic/train.csv')

In [4]:
df.groupby('Destination').count()
df.groupby('HomePlanet').count()
print('Cabin의 개수',len(df.groupby('Cabin').count()))

Cabin의 개수 6560


컬럼
- Destination 목적지 {55 Cancri e, PSO J318.5-22, TRAPPIST-1e}
- HomePlanet  {Earth , Europa, Mars}
- Cabin의 종류 6560
    - Cabin을 앞 대문자만 남기면 의미가 있을 수 있을까?
    - 돈이 많은 사람이 사는 자리 등으로 의미가 있는지확인
    - A,B,C,D,E,F,G,T로 변환 후 다시 숫자로 0~7까지 변환
- True / False 
    - CryoSleep
    - VIP
    - Transported(target)
- 삭제
    - PassengerId
    - Name

In [7]:
# 범주형 데이터 변환
def df_replace(dataframe):

    global df_train
    df = dataframe
    df = df.replace({'55 Cancri e':0,'PSO J318.5-22':1,'TRAPPIST-1e':2})
    df = df.replace({'Earth':0,'Europa':1,'Mars':2})
    df = df.replace({False:0,True:1})

    df['Cabin'] = df['Cabin'].str[0]
    df = df.replace({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7})


    ## 범주형 데이터 확인할 수 있게 dict로 만들어 놓은 것
    Cabin = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7}
    Destination = {'55 Cancri e': 0, 'PSO J318.5-22': 1,'TRAPPIST-1e':2}
    HomePlanet = {'Earth':0,'Europa':1,'Mars':2}
    CryoSleep = {'False':0,'True':1}
    VIP = {'False':0,'True':1}
    Transported = {'False':0,'True':1}


    df_train = df.drop(['PassengerId','Name'],axis=1)
    # df 전체 float형으로 바꾸기
    df_train = df_train.astype(float)

df_replace(df)

In [8]:
df_train

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1.0,0.0,1.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,5.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1.0
2,1.0,0.0,0.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0.0
3,1.0,0.0,0.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0.0
4,0.0,0.0,5.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,1.0,0.0,0.0,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0.0
8689,0.0,1.0,6.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8690,0.0,0.0,6.0,2.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1.0
8691,1.0,0.0,4.0,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0.0


### null값 처리
XGBOOST를 사용해서 pridct값을 null값으로 대체

In [9]:
# df_homeplanet =  df_train중에서 HomePlanet의 null값이 존재하는 열만 추출 
# -> 훈련된 모델의 값으로 다시 채울 예정
df_homeplanet = df_train[df_train['HomePlanet'].isnull()]

#df_train 중에서 HomePlanet의 null값이 존재하는 열만 추출 -> 추후 훈련할 데이터프레임
df_train01 = df_train.dropna(subset=['HomePlanet'])
# df_train
df_train[df_train['HomePlanet'].isnull()]

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
59,NaN,1.0,4.0,2.0,33.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0
113,NaN,0.0,0.0,2.0,39.0,0.0,0.0,2344.0,0.0,65.0,6898.0,0.0
186,NaN,1.0,3.0,0.0,24.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0
225,NaN,0.0,5.0,2.0,18.0,0.0,313.0,1.0,691.0,283.0,0.0,0.0
234,NaN,1.0,2.0,0.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8515,NaN,0.0,4.0,2.0,25.0,0.0,1258.0,0.0,22.0,19.0,0.0,0.0
8613,NaN,0.0,4.0,0.0,53.0,0.0,0.0,4017.0,0.0,13.0,3147.0,0.0
8666,NaN,0.0,5.0,0.0,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,1.0
8674,NaN,0.0,5.0,2.0,13.0,0.0,39.0,0.0,1085.0,24.0,0.0,0.0


In [10]:
# train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train01.drop('HomePlanet',axis=1),df_train01['HomePlanet'], test_size=0.2, random_state=42)

# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_train.shape
y_train.shape
X_test.shape
y_test.shape

(1699,)

In [11]:
#XGBoost1
import xgboost as xgb
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = xgb.XGBClassifier(early_stopping_rounds=10)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

pred = model.predict(X_test)
acc = accuracy_score(y_test,pred)
acc

# from sklearn.metrics import confusion_matrix
# import scikitplot as skplt

# pred = model.predict(X_test)
# # 실제 , 예측
# cm = confusion_matrix(y_test, pred)
# print('confusion_matrix : \n', cm)

# skplt.metrics.plot_confusion_matrix(y_test,pred,figsize=(8,6))
# plt.show()

# cl_report = metrics.classification_report(y_test,pred)
# print('리포트:\n',cl_report)

0.9387875220718069

In [225]:
## df_01 은 df_train 중에서 HomePlanet의 값이 null인 경우의 데이터프레임
df_01 = df_homeplanet.drop('HomePlanet',axis=1)
df_01

pred = model.predict(df_01)
pred


df_homeplanet['HomePlanet'] = pred
df_homeplanet
df_update = pd.concat([df_train01,df_homeplanet],axis=0)
df_update

,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
59,1.0,4.0,2.0,33.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0
113,0.0,0.0,2.0,39.0,0.0,0.0,2344.0,0.0,65.0,6898.0,0.0
186,1.0,3.0,0.0,24.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0
225,0.0,5.0,2.0,18.0,0.0,313.0,1.0,691.0,283.0,0.0,0.0
234,1.0,2.0,0.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
8515,0.0,4.0,2.0,25.0,0.0,1258.0,0.0,22.0,19.0,0.0,0.0
8613,0.0,4.0,0.0,53.0,0.0,0.0,4017.0,0.0,13.0,3147.0,0.0
8666,0.0,5.0,0.0,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,1.0
8674,0.0,5.0,2.0,13.0,0.0,39.0,0.0,1085.0,24.0,0.0,0.0


array([2, 1, 1, 2, 1, 0, 0, 2, 0, 1, 2, 0, 2, 1, 2, 1, 0, 2, 0, 2, 1, 1,
       2, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 2, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 1, 0, 2, 0, 0, 2, 2, 0, 0, 0, 1, 0, 0, 0, 2,
       2, 1, 0, 1, 0, 0, 1, 0, 2, 0, 0, 1, 0, 1, 2, 2, 0, 2, 2, 0, 0, 0,
       0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 1, 2, 0, 1, 2, 2, 0, 0, 0, 1, 0, 0,
       2, 1, 1, 0, 0, 0, 2, 0, 0, 1, 1, 0, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 2, 2, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 1, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 1,
       0, 2, 0])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
59,2,1.0,4.0,2.0,33.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0
113,1,0.0,0.0,2.0,39.0,0.0,0.0,2344.0,0.0,65.0,6898.0,0.0
186,1,1.0,3.0,0.0,24.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0
225,2,0.0,5.0,2.0,18.0,0.0,313.0,1.0,691.0,283.0,0.0,0.0
234,1,1.0,2.0,0.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8515,2,0.0,4.0,2.0,25.0,0.0,1258.0,0.0,22.0,19.0,0.0,0.0
8613,1,0.0,4.0,0.0,53.0,0.0,0.0,4017.0,0.0,13.0,3147.0,0.0
8666,0,0.0,5.0,0.0,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,1.0
8674,2,0.0,5.0,2.0,13.0,0.0,39.0,0.0,1085.0,24.0,0.0,0.0


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1.0,0.0,1.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,5.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1.0
2,1.0,0.0,0.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0.0
3,1.0,0.0,0.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0.0
4,0.0,0.0,5.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8515,2.0,0.0,4.0,2.0,25.0,0.0,1258.0,0.0,22.0,19.0,0.0,0.0
8613,1.0,0.0,4.0,0.0,53.0,0.0,0.0,4017.0,0.0,13.0,3147.0,0.0
8666,0.0,0.0,5.0,0.0,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,1.0
8674,2.0,0.0,5.0,2.0,13.0,0.0,39.0,0.0,1085.0,24.0,0.0,0.0


In [229]:
df_update[df_update['HomePlanet'].isnull()]

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
